In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.8 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset


In [4]:
from datasets import load_dataset

dataset = load_dataset("wmt14", "de-en")  # WMT-14 English-German
train_data = dataset["train"]
valid_data = dataset["validation"]
test_data = dataset["test"]


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/280M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/273M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/474k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [14]:
train_data

Dataset({
    features: ['translation'],
    num_rows: 10
})

In [26]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Extract English and German sentences
corpus = [example["translation"]["en"] for example in train_data] + \
         [example["translation"]["de"] for example in train_data]

# Train the tokenizer on the corpus
trainer = trainers.BpeTrainer(special_tokens=["<s>", "</s>", "<pad>", "<unk>"], vocab_size=32000)
tokenizer.train_from_iterator(corpus, trainer=trainer)

# Save the tokenizer for future use
tokenizer.save("bpe_tokenizer.json")


In [28]:
def tokenize_function(example):
    # Tokenize English and German text
    src_tokens = tokenizer.encode(example["translation"]["en"]).ids
    tgt_tokens = tokenizer.encode(example["translation"]["de"]).ids
    return {"src_tokens": src_tokens, "tgt_tokens": tgt_tokens}

# Apply tokenization
train_data = train_data.map(tokenize_function, remove_columns=["translation"])
# valid_data = valid_data.map(tokenize_function, remove_columns=["translation"])
# test_data = test_data.map(tokenize_function, remove_columns=["translation"])


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [29]:
train_data

Dataset({
    features: ['src', 'tgt', 'src_tokens', 'tgt_tokens'],
    num_rows: 10
})

In [31]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

# Define a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Create PyTorch DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=data_collator)
valid_loader = DataLoader(valid_data, batch_size=32, collate_fn=data_collator)
test_loader = DataLoader(test_data, batch_size=32, collate_fn=data_collator)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [32]:
train_loader

In [33]:
import torch.nn as nn

# Define the embedding layer
vocab_size = tokenizer.get_vocab_size()
d_model = 512  # Dimension of embeddings
embedding = nn.Embedding(vocab_size, d_model)

# Example usage
batch = next(iter(train_loader))  # A batch from the dataloader
src_tokens = batch["src_tokens"]  # Source token IDs
tgt_tokens = batch["tgt_tokens"]  # Target token IDs

src_embeddings = embedding(src_tokens)  # Shape: (batch_size, seq_length, d_model)
tgt_embeddings = embedding(tgt_tokens)  # Shape: (batch_size, seq_length, d_model)


AttributeError: 'tokenizers.Tokenizer' object has no attribute 'pad'

In [2]:
import math
import torch
import numpy as np


def softmax(x: torch.Tensor, d: int):
    # return torch.nn.Softmax(dim=d)(x)
    return torch.nn.functional.softmax(x, dim=d)


class FeedForward(torch.nn.Module):
    def __init__(
        self,
        d_model: int = 512,
        hidden_size: list[tuple[int]] = None,
        activation: torch.nn.Module = torch.nn.ReLU(),
    ):
        """
        notes:
            - default dimensions are set according to paper

        """
        if hidden_size is None:
            hidden_size = [(d_model, 2048), (2048, d_model)]

        assert len(hidden_size) > 0, "hidden_size must be greater than 0"
        assert (
            hidden_size[0][0] == d_model and hidden_size[-1][1] == d_model
        ), "input and output dimensions must equal d_model"

        super().__init__()

        layers = []
        n = len(hidden_size)
        for i in range(n):
            layers.append(torch.nn.Linear(*hidden_size[i]))

            # No activation after the final layer
            if i < n - 1:
                layers.append(activation)

        self.net = torch.nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        return self.net(x)


def attention(
    q: torch.Tensor,
    k: torch.Tensor,
    v: torch.Tensor,
    pad_mask: torch.Tensor,
    mask: torch.Tensor = None,
):
    """
    input:
        q: a torch tensor of size: (batch_size*h, seq_len, d_k)
        k: a torch tensor of size: (batch_size*h, seq_len, d_k)
        v: a torch tensor of size: (batch_size*h, seq_len, d_v)
        pad_mask: a torch tensor of size: (batch_size*h, seq_len, seq_len)
        mask: a torch tensor of size: (seq_len, seq_len)

    output:
        A torch tensor of size: (batch_size*h, seq_len, d_v)

    """
    x = torch.matmul(q, torch.transpose(k, 1, 2))

    # Scale x by sqrt(d_k)
    x = x / math.sqrt(q.size(2))

    if mask is not None:

        # Broadcasting will match the size at dim=0
        mask = mask.unsqueeze(0)

        # Apply mask via element-wise addition
        x = x + mask

    x = x + pad_mask

    return torch.matmul(softmax(x, d=2), v)


class MultiHead(torch.nn.Module):
    def __init__(self, h: int = 8, d_model: int = 512, mask: torch.Tensor = None):
        """
        input:
            h: number of heads
            d_model: model dimensions, i.e. embedding size
            mask: a boolean to apply masked multi-head attention
        notes:
            - default dimensions are set according to the paper

        """
        assert d_model % h == 0, "d_model must be divisible by h"
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // h
        self.d_v = self.d_k
        self.h = h
        self.mask = mask

        self.w_q = torch.nn.Linear(d_model, d_model)
        self.w_k = torch.nn.Linear(d_model, d_model)
        self.w_v = torch.nn.Linear(d_model, d_model)
        self.w_o = torch.nn.Linear(d_model, d_model)

    def forward(
        self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, pad_mask: torch.Tensor
    ):
        """
        input:
            q: a torch tensor of size: (batch_size, seq_len, d_model)
            k: a torch tensor of size: (batch_size, seq_len, d_model)
            v: a torch tensor of size: (batch_size, seq_len, d_model)
            pad_mask: a torch tensor of size: (batch_size, seq_len, seq_len)

        output:
            A torch tensor of size: (batch_size, seq_len, d_model)

        notes:
            - d_model is essentially embedding dimensions
        """
        batch_size, seq_len, _ = q.size()
        q_h = self.w_q(q)
        k_h = self.w_k(k)
        v_h = self.w_v(v)

        # Splitting q, k and v tensors in to h heads
        q_h = q_h.reshape(batch_size, seq_len, self.h, self.d_k).transpose(1, 2)
        k_h = k_h.reshape(batch_size, seq_len, self.h, self.d_k).transpose(1, 2)
        v_h = v_h.reshape(batch_size, seq_len, self.h, self.d_v).transpose(1, 2)

        # Combine heads for parrallel computation
        q_h = q_h.reshape(batch_size * self.h, seq_len, self.d_k)
        k_h = k_h.reshape(batch_size * self.h, seq_len, self.d_k)
        v_h = v_h.reshape(batch_size * self.h, seq_len, self.d_v)

        # Apply attention
        scores = attention(q_h, k_h, v_h, pad_mask, mask=self.mask)

        # Seperate heads
        scores = scores.reshape(batch_size, self.h, seq_len, self.d_v).transpose(1, 2)

        # Concat h heads (Concat(head1, ..., headh))
        scores = scores.reshape(batch_size, seq_len, self.h * self.d_v)

        return self.w_o(scores)


# TODO: can change the definition so that if dropout called with
# p=None then can just terminate. This would make the Encoder and
# Decoder forward passes more concise
def dropout(x: torch.Tensor, p: float = 0.1):
    assert 0 <= p <= 1, "p must be a probability"

    if p == 1:
        # All elements are dropped; just return zeros.
        return torch.zeros_like(x)

    # Create a tensor with the same shape as x
    # and set all is values to 1 - p
    mask = torch.full_like(x, 1 - p)

    # Will sample the entries from the bernoulli distribution.
    # The i'th entry of the output tensor will draw a value 1 according
    # to the i'th probability given the input tensor.
    mask = torch.bernoulli(mask).to(x.device)

    # Apply dropout via element-wise multiplication.
    x = x * mask

    # Apply inverted scaling
    return x * (1 / (1 - p))


class Encoder(torch.nn.Module):
    def __init__(self, number_of_heads: int = 8, d_model: int = 512):
        """
        notes:
            - default parameter values are based on the paper
        """
        assert (
            d_model % number_of_heads == 0
        ), "d_model must be divisible by number_of_heads"
        super().__init__()
        self.multi_head_attention = MultiHead(h=number_of_heads, d_model=d_model)
        self.feed_forward = FeedForward(d_model=d_model)
        self.d_model = d_model
        self.layer_norms = torch.nn.ModuleList(
            torch.nn.LayerNorm(d_model) for _ in range(2)
        )

    def forward(
        self, x: torch.Tensor, pad_mask: torch.Tensor, dropout_p=None
    ) -> torch.Tensor:
        """
        input:
            x: a torch tensor of size: (batch_size, seq_len, d_model)
        output:
            a torch tensor of size: (batch_size, seq_len, d_model)
        """

        x = self.layer_norms[0](
            x
            + (
                self.multi_head_attention(x, x, x, pad_mask)
                if dropout_p is None
                else dropout(self.multi_head_attention(x, x, x, pad_mask), p=dropout_p)
            )
        )

        return self.layer_norms[1](
            x
            + (
                self.feed_forward(x)
                if dropout_p is None
                else dropout(self.feed_forward(x), p=dropout_p)
            )
        )


class Decoder(torch.nn.Module):
    def __init__(
        self,
        attention_mask: torch.Tensor,
        number_of_heads: int = 8,
        d_model: int = 512,
    ):
        """
        notes:
            - default parameter values are based on the paper
            - in the previous implementation we were using the same multi head
            module and inidicating the apply mask on the forward pass. That's an
            outregous mistake. The same parameters are being used in that case
            only when applying attention part of the sequence was getting masked.
        """
        assert (
            d_model % number_of_heads == 0
        ), "d_model must be divisible by number_of_heads"
        super().__init__()
        self.multi_head_attention = MultiHead(h=number_of_heads, d_model=d_model)
        self.masked_multi_head_attention = MultiHead(
            h=number_of_heads, d_model=d_model, mask=attention_mask
        )
        self.feed_forward = FeedForward(d_model=d_model)
        self.d_model = d_model
        self.layer_norms = torch.nn.ModuleList(
            torch.nn.LayerNorm(d_model) for _ in range(3)
        )

    def forward(
        self,
        x: torch.Tensor,
        encoder_x: torch.Tensor,
        decoder_pad_mask: torch.Tensor,
        encoder_pad_mask: torch.Tensor,
        dropout_p=None,
    ) -> torch.Tensor:
        """
        inputs:
            x: a torch tensor of size: (batch_size, seq_len, d_model)
            encoder_x: a torch tensor of size: (batch_size, seq_len, d_model)
        output:
            a torch tensor of size: (batch_size, seq_len, d_model)
        """
        x = self.layer_norms[0](
            x
            + (
                self.masked_multi_head_attention(x, x, x, decoder_pad_mask)
                if dropout_p is None
                else dropout(
                    self.masked_multi_head_attention(x, x, x, decoder_pad_mask),
                    p=dropout_p,
                )
            )
        )

        # Cross-Attention
        # Here encoder_pad_mask must be used because
        x = self.layer_norms[1](
            x
            + (
                self.multi_head_attention(x, encoder_x, encoder_x, encoder_pad_mask)
                if dropout_p is None
                else dropout(
                    self.multi_head_attention(
                        x, encoder_x, encoder_x, encoder_pad_mask
                    ),
                    p=dropout_p,
                )
            )
        )

        return self.layer_norms[2](
            x
            + (
                self.feed_forward(x)
                if dropout_p is None
                else dropout(self.feed_forward(x), p=dropout_p)
            )
        )


def positional_encodings(seq_len: int, d_model: int) -> np.ndarray:
    pos, i = np.indices((seq_len, d_model))
    return np.where(
        i % 2 == 0,
        np.sin(pos / np.power(10000, (2 * i / d_model))),
        np.cos(pos / np.power(10000, (2 * i / d_model))),
    )


def get_attention_mask(seq_len: int, dtype: torch.dtype) -> torch.Tensor:
    """
    Here we will be creating a triangular matrix where
    all the upper triangle (above the diagonal) is set to -oo.

    output:
        A torch tensor of size: (seq_len, seq_len)
    """

    mask_x = torch.full((seq_len, seq_len), float("-inf"), dtype=dtype)

    return torch.triu(mask_x, diagonal=1)

    # Add a new batch_size dimension and expand it to
    # match batch_size
    # -1 means keep the size at that dimension
    # However its also documented here:
    # https://stackoverflow.com/questions/65900110/does-pytorch-broadcast-consume-less-memory-than-expand
    # that expand does not also consume extra memory
    # mask = mask.unsqueeze(0).expand(batch_size, -1, -1)


def get_pad_mask(
    x: torch.Tensor, h: int, batch_size: int, seq_len: int, pad_token_id: int = 0
) -> torch.Tensor:
    """
    inputs:
        x: a torch tensor of size: (batch_size, seq_len)
    """
    # 0 must be the padding token id
    # Here torch.where(encoder_x == 0, float("-inf"), 0)
    #    .unsqueeze(2)
    #    .expand(-1, -1, self.seq-len)
    # should also mathematically produce the same shape
    # but typically the keys gets masked so the current approach
    # aligns better with the paper
    assert x.size() == (
        batch_size,
        seq_len,
    ), f"x must have size: {(batch_size, seq_len)}"

    return (
        torch.where(x == pad_token_id, float("-inf"), 0)
        .unsqueeze(1)
        .unsqueeze(1)
        .expand(-1, h, seq_len, -1)
        .reshape(batch_size * h, seq_len, seq_len)
    )


class Transformer(torch.nn.Module):
    def __init__(
        self,
        batch_size: int,
        seq_len: int,
        d_model: int = 512,
        n: int = 6,
        number_of_heads: int = 8,
        input_vocab_size: int = 37000,
        output_vocab_size: int = 37000,
        dtype: torch.dtype = torch.float32,
        dropout_p: float = None,
    ):
        """
        inputs:
            n: the number of encoder and decoder stacks
            d_model: model dimensions, i.e. embedding dimensions
            number_of_heads: number of heads using in multi-head attention
            input_vocab_size: size of the vocabulary for the input structure
            output_vocab_size: size of the vocabulary for the output structure

        notes:
            - I'm using a fixed seq_len for both the input and the output. That could
            be adjusted to make it varied and more flexible.

        """
        super().__init__()
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.n = n
        self.d_model = d_model
        self.number_of_heads = number_of_heads
        self.vocab_size = output_vocab_size
        self.dtype = dtype
        self.register_buffer(
            "pe",
            torch.from_numpy(positional_encodings(seq_len, d_model))
            .to(dtype)
            .unsqueeze(0),
        )
        self.register_buffer("attention_mask", get_attention_mask(seq_len, dtype=dtype))
        self.linear = torch.nn.Linear(d_model, output_vocab_size)
        assert (
            dropout_p is None or 0 <= dropout_p <= 1
        ), "p_dropout must be a value between 0 and 1"
        self.dropout_p = dropout_p

        self.input_embedding = torch.nn.Embedding(
            num_embeddings=input_vocab_size, embedding_dim=d_model
        )
        self.output_embedding = torch.nn.Embedding(
            num_embeddings=output_vocab_size, embedding_dim=d_model
        )

        # Using ModuleList is crucial here instead of python list
        # If python list is used, then model.parameters() will not
        # return the paremeters of layers inside.
        self.encoders = torch.nn.ModuleList(
            Encoder(number_of_heads=number_of_heads, d_model=d_model)
            for _ in range(self.n)
        )

        self.decoders = torch.nn.ModuleList(
            Decoder(
                attention_mask=self.attention_mask,
                number_of_heads=number_of_heads,
                d_model=d_model,
            )
            for _ in range(self.n)
        )

    def forward(
        self,
        encoder_x: torch.Tensor,
        decoder_x: torch.Tensor,
        apply_softmax: bool = False,
        pad_token_id: int = 0,
    ):
        """
        inputs:
            encoder_x: a torch tensor of size: (batch_size, seq_len)
            decoder_x: a torch tensor of size: (batch_size, seq_len)
            apply_softmax: a boolean.
                - Most torch loss functions expect logits instead of porbabilities.
                So make sure that the loss function does not normalize inputs and
                expects probabilities before setting this to True.

        outputs:
            a probability distribution over the vocabulary if apply_softmax is true else
            it outputs logits
        """
        assert (
            encoder_x.size() == decoder_x.size() == (self.batch_size, self.seq_len)
        ), f"encoder_x and decoder_x must both have the size: ({self.batch_size}, {self.seq_len})"

        assert torch.any(encoder_x) and torch.any(decoder_x), (
            "empty examples are not allowed. There could be some additional reasons "
            "for not to allow them but simple example why is that softmax is not defined "
            "over an empty sequence"
        )

        encoder_x_pad_mask = get_pad_mask(
            encoder_x,
            h=self.number_of_heads,
            batch_size=self.batch_size,
            seq_len=self.seq_len,
            pad_token_id=pad_token_id,
        )
        decoder_x_pad_mask = get_pad_mask(
            decoder_x,
            h=self.number_of_heads,
            batch_size=self.batch_size,
            seq_len=self.seq_len,
            pad_token_id=pad_token_id,
        )

        # In the paper it is mentioned that they scale the embedding weights by math.sqrt(self.d_model)
        # see the end of section 3.4 for more detail
        encoder_x = self.input_embedding(encoder_x) * math.sqrt(self.d_model) + self.pe
        decoder_x = self.output_embedding(decoder_x) * math.sqrt(self.d_model) + self.pe

        for encoder in self.encoders:
            encoder_x = encoder(encoder_x, encoder_x_pad_mask, dropout_p=self.dropout_p)

        for decoder in self.decoders:
            decoder_x = decoder(
                decoder_x,
                encoder_x,
                decoder_pad_mask=decoder_x_pad_mask,
                encoder_pad_mask=encoder_x_pad_mask,
                dropout_p=self.dropout_p,
            )

        return (
            softmax(self.linear(decoder_x), d=2)
            if apply_softmax
            else self.linear(decoder_x)
        )

In [13]:
import torch
import numpy as np
from torch.utils.data import DataLoader

# Use the CountingDataset
class CountingDataset(torch.utils.data.Dataset):
    def __init__(self, n, max_length=8, vocab_size=3):
        assert vocab_size > 2
        self.n = n
        self.vocab_size = vocab_size
        seq_lengths = np.random.randint(max_length // 2, max_length, n)
        data = np.random.randint(1, vocab_size, (n, max_length))

        # Replace elements past the sequence length with 0 (padding token)
        for i in range(n):
            data[i, seq_lengths[i]:] = 0

        # Labels: whether `1`s outnumber `2`s
        num_ones = (data == 1).sum(axis=1)
        num_twos = (data == 2).sum(axis=1)
        self.data = torch.tensor(data, dtype=torch.long)
        self.labels = torch.tensor((num_ones > num_twos).astype(int), dtype=torch.long)

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# Create dataset and dataloader
dataset = CountingDataset(1000, max_length=8, vocab_size=3)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

# Transformer Model Parameters
batch_size = 32
seq_len = 8
d_model = 16
number_of_heads = 2
n = 2
input_vocab_size = 3  # Matches CountingDataset
output_vocab_size = 2  # Binary classification (0 or 1)

# Instantiate Transformer Model
model = Transformer(
    batch_size=batch_size,
    seq_len=seq_len,
    d_model=d_model,
    n=n,
    number_of_heads=number_of_heads,
    input_vocab_size=input_vocab_size,
    output_vocab_size=output_vocab_size,
)

# Define Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 120
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x, y in dataloader:
        # Initialize decoder input with start-of-sequence tokens
        decoder_x = torch.zeros_like(x)
        decoder_x[:, 0] = 1  # Start-of-sequence token

        # Forward pass
        output = model(x, decoder_x)

        # Use output at the last position for classification
        logits = output[:, -1, :]  # Shape: (batch_size, output_vocab_size)
        loss = criterion(logits, y)

        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}")

# Testing the Model
model.eval()
with torch.no_grad():
    for test_data, test_labels in dataloader:
        if test_data.size(0) != batch_size:
            continue

        decoder_x = torch.zeros_like(test_data)
        decoder_x[:, 0] = 1  # Start-of-sequence token
        output = model(test_data, decoder_x, apply_softmax=True)
        predictions = torch.argmax(output[:, -1, :], dim=-1)

        print("Test Labels:", test_labels.numpy())
        print("Predictions:", predictions.numpy())
        break


Epoch 1/120, Loss: 0.28974135328204403
Epoch 2/120, Loss: 0.08789960349038724
Epoch 3/120, Loss: 0.026533257636812427
Epoch 4/120, Loss: 0.016345204274740913
Epoch 5/120, Loss: 0.011934845978694578
Epoch 6/120, Loss: 0.009116222931733054
Epoch 7/120, Loss: 0.007177581755264151
Epoch 8/120, Loss: 0.005799303402102763
Epoch 9/120, Loss: 0.004775385687788648
Epoch 10/120, Loss: 0.004000444964115177
Epoch 11/120, Loss: 0.0033937536946107303
Epoch 12/120, Loss: 0.0029148882435214136
Epoch 13/120, Loss: 0.002529515937391308
Epoch 14/120, Loss: 0.0022183142362102387
Epoch 15/120, Loss: 0.0019572653597401033
Epoch 16/120, Loss: 0.0017383652739226818
Epoch 17/120, Loss: 0.001551750345125554
Epoch 18/120, Loss: 0.0013946275407027814
Epoch 19/120, Loss: 0.0012597408213262115
Epoch 20/120, Loss: 0.0011412799884114534
Epoch 21/120, Loss: 0.001037704240110132
Epoch 22/120, Loss: 0.0009479788150788556
Epoch 23/120, Loss: 0.0008678083371881756
Epoch 24/120, Loss: 0.0007980144319815501
Epoch 25/120, Lo

In [2]:
import time

import numpy as np
import torch


import os
import torch


def save(filename, **kwargs):
    """
    Save a pytorch object to file
    See: https://pytorch.org/tutorials/beginner/saving_loading_models.html

    Arguments:
        filename: the file in which to save the object

    Possible keyword arguments (kwargs):
        epoch: the epoch so far if training
        model_state_dict: a model's state
        opt_state_dict: a optimizer's state, if training
    """

    msg = f"{filename} exists: delete it first to replace it."
    assert not os.path.exists(filename), msg
    torch.save(kwargs, filename)


def load(filename):
    """
    Load a pytorch object from a given filename
    See: https://pytorch.org/tutorials/beginner/saving_loading_models.html
    You shouldn't need to edit this function.

    Arguments:
        filename: the file from which to load the object
    """

    return torch.load(filename)


class Trainer:
    def __init__(self, optimizer, model, loss_func, scheduler=None, **kwargs):
        """
        Initialize the optimizer for the model, using any necessary kwargs
        Save the model and loss function for later calculation
        You shouldn't need to edit this function.
        """

        self.optimizer: torch.optim.Optimizer = optimizer(model.parameters(), **kwargs)
        self.scheduler = scheduler["scheduler"](
            self.optimizer, lr_lambda=scheduler["lr_lambda"]
        )
        self.model: torch.nn.Module = model
        self.loss_func = loss_func

        self.epoch = 0
        self.start_time = None

    def run_one_batch(self, encder_x, decoder_x, y, train=True, pad_token_id=0):
        """
        Run self.model on one batch of data, using `self.loss_func` to
            compute the model's loss.

        If train=True (the default), you should use `self.optimizer`
            to update the parameters of `self.model`.

        You should also call `self.optimizer.zero_grad()`; see
            https://pytorch.org/tutorials/recipes/recipes/zeroing_out_gradients.html
            for a guide as to when to do that.

        Args
            enocder_x: the batch's enocder input
            decoder_x: the batch's decoder input
            y: the batch's target

        Returns
            loss: the model's loss on this batch
        """

        if train:
            self.optimizer.zero_grad()

        outputs = self.model(encder_x, decoder_x, pad_token_id=pad_token_id)
        loss = self.loss_func(outputs.transpose(1, 2), y)

        if train:
            loss.backward()
            self.optimizer.step()
            if self.scheduler is not None:
                self.scheduler.step()

        return loss.detach().numpy()

    def run_one_epoch(
        self,
        data_loader: torch.utils.data.DataLoader,
        train=True,
        verbose=False,
        pad_token_id=0,
    ):
        """
        Train one epoch, a batch at a time, using self.run_one_batch
        You shouldn't need to edit this function.

        Args:
            data_loader: a torch.utils.data.DataLoader with our dataset
            stats: an optional dict of information to print out

        Returns:
            total_loss: the average loss per example
        """
        np.random.seed(0)
        torch.manual_seed(0)
        # torch.use_deterministic_algorithms(True)
        if self.start_time is None:
            self.start_time = time.time()

        epoch_size = 0
        total_loss = 0
        for batch_idx, batch_data in enumerate(data_loader):
            encoder_x, decoder_x, y = (
                batch_data["input_ids"],
                batch_data["decoder_input_ids"],
                batch_data["labels"],
            )
            epoch_size += encoder_x.size(0)
            loss = self.run_one_batch(
                encoder_x, decoder_x, y, train=train, pad_token_id=pad_token_id
            )
            total_loss += loss

        avg_loss = total_loss / epoch_size

        if verbose:
            epoch = self.epoch + 1
            duration = (time.time() - self.start_time) / 60

            if train:
                log = [f"Epoch: {epoch:6d}"]
            else:
                log = ["Eval:" + " " * 8]

            log.extend(
                [
                    f"Loss: {avg_loss:6.3f}",
                    f"in {duration:5.1f} min",
                ]
            )
            print("  ".join(log))

        return avg_loss

    def train(
        self, data_loader, n_epochs, train=True, report_every=None, pad_token_id=0
    ):
        """
        Run the model for `n_epochs` epochs on the data in `data_loader`
        You shouldn't need to edit this function.

        Args
            data_loader: data loader for our data
            n_epochs: how many epochs to run
            train: if True, train the model; otherwise, just evaluate it
            report_every: how often to print out stats

        Returns
            losses: average loss per epoch
        """
        self.start_time = time.time()

        if report_every is None:
            report_every = max(1, n_epochs // 10)

        losses = []
        for i in range(n_epochs):
            verbose = ((i + 1) % report_every) == 0
            loss = self.run_one_epoch(
                data_loader, train=train, verbose=verbose, pad_token_id=pad_token_id
            )
            losses.append(loss)
            if train:
                self.epoch += 1

        return losses

    def eval(self, data_loader):
        """
        Helper function to run through the data loader once and just
            compute the loss
        You shouldn't need to edit this function.
        """
        return self.train(data_loader, 1, train=False, report_every=1)

    def save_trainer(self, filename):
        """
        Use `src.utils.save` to save this Trainer to file.
        See https://pytorch.org/tutorials/beginner/saving_loading_models.html

        Args
            filename: the file to which to save the trainer
        """
        save(
            filename,
            epoch=self.epoch,
            model_state_dict=self.model.state_dict(),
            optimizer_state_dict=self.optimizer.state_dict(),
        )

    def load_trainer(self, filename):
        """
        Use `src.utils.load` to load this trainer from file.
        See https://pytorch.org/tutorials/beginner/saving_loading_models.html

        Note: in addition to simply loading the saved model, you must
            use the information from that checkpoint to update the model's
            state.

        Args
            filename: the file from which to load the model
        """
        checkpoint = load(filename)
        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        self.epoch = checkpoint["epoch"]

        # this ensures that the trainer's/model's parameters are in training mode
        self.model.train()

In [3]:
"""
Loads the wmt12 german-to-englis (de-en) variable from huggging 
face or cache (if its already downloads before)
and stores it in a variable called wmt14

"""

import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset

wmt14 = load_dataset("wmt14", "de-en")

# Display available splits
print(wmt14)

# Display column names and data types for the 'train' split
print(wmt14["train"].features)

# Display the first 5 rows of the 'train' split
for i in range(5):
    print(wmt14["train"][i])

# # Convert the 'train' split to a DataFrame
# df_train = pd.DataFrame(wmt14["train"])

# # Display the first 5 rows
# print(df_train.head())

# # Calculate sentence lengths
# en_lengths = [len(ex["translation"]["en"].split()) for ex in wmt14["train"]]
# de_lengths = [len(ex["translation"]["de"].split()) for ex in wmt14["train"]]

# # Plot histograms
# plt.hist(en_lengths, bins=50, alpha=0.5, label="English")
# plt.hist(de_lengths, bins=50, alpha=0.5, label="German")
# plt.xlabel("Sentence Length")
# plt.ylabel("Frequency")
# plt.legend(loc="upper right")
# plt.title("Sentence Length Distribution")
# plt.show()

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/280M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/273M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/474k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})
{'translation': Translation(languages=['de', 'en'], id=None)}
{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}
{'translation': {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.', 'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'}}
{'translation': {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht 

In [4]:
"""
This module provides functionality to preprocess and tokenize examples from 
the WMT14 dataset for training a transformer model.

The WMT14 dataset is a collection of parallel sentences in German and English. 
The dataset is structured as follows:

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})

Each example in the dataset is a dictionary with a 'translation' key, which 
contains a dictionary with 'de' (German) and 'en' (English) keys:

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}
{'translation': {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode 
des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel 
und hoffe, daß Sie schöne Ferien hatten.', 'en': 'I declare resumed the session of the European Parliament 
adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope 
that you enjoyed a pleasant festive period.'}}

This module converts these examples into tokenized inputs and decoder inputs suitable for 
training a transformer model. The preprocessing steps include:

1. Loading a pre-trained English-German tokenizer.
2. Determining the maximum sequence length for tokenized sequences.
3. Defining a preprocessing function to tokenize the source (German) 
and target (English) sentences.
4. Applying the preprocessing function to a subset of the training set.
5. Converting the tokenized examples to PyTorch tensors for consistency.

Example output after preprocessing:
Converts examples from the WMT14 dataset to tokenized inputs and decoder inputs

# Example
Example 0
Input IDs: tensor([423, 42, 11967, 2985, 2, 29, 9, 372, 4209, 8319])
Decoder Input IDs: tensor([4677, 2, 52, 41, 73, 72, 1770, 2, 4, 48186])
Decoded decoder input: Although, as you will have seen, the dreaded 
'millennium bug' failed to materialise...

"""

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Pre-trained English-German tokenizer/model
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

# print("Tokenizer pad token:", tokenizer.pad_token)  # prints: '<pad>'
# print("Tokenizer pad token id:", tokenizer.pad_token_id)  # prints: '58100'


# Determine max length of tokenized sequences before padding
# def length_check_function(examples):
#     # Temporarily no truncation or padding
#     src_lengths, tgt_lengths = [], []
#     for translation in examples["translation"]:
#         src_lengths.append(len(tokenizer.tokenize(translation["de"])))
#         tgt_lengths.append(len(tokenizer.tokenize(translation["en"])))

#     return {"src_length": src_lengths, "tgt_length": tgt_lengths}


# lengths = wmt14["train"].map(length_check_function, batched=True)
# max_source_length = max(lengths["src_length"])
# max_target_length = max(lengths["tgt_length"])
# max_seq_len = max(max_source_length, max_target_length)
# print("Max sequence length:", max_seq_len)  # prints: '13,614'

max_seq_len = 32


if tokenizer.bos_token is None:
    tokenizer.add_special_tokens({"bos_token": "<s>"})
    # If a model is already loaded, you may need:
    model.resize_token_embeddings(len(tokenizer))


def preprocess_function(examples):
    # Tokenize source (German)
    model_inputs = tokenizer(
        [translation["de"] for translation in examples["translation"]],
        max_length=max_seq_len,
        padding="max_length",
        truncation=True,
    )

    # Tokenize target (English)
    with tokenizer.as_target_tokenizer():
        tokenized_targets = tokenizer(
            [translation["en"] for translation in examples["translation"]],
            max_length=max_seq_len,
            padding="max_length",
            truncation=True,
        )

    input_ids = tokenized_targets["input_ids"]

    # Ensure bos_token_id is defined (after adding bos_token if needed)
    bos_id = tokenizer.bos_token_id
    eos_id = tokenizer.eos_token_id

    # Create decoder_input_ids by prepending BOS and removing the last token
    # Example:
    # original input_ids: [w1, w2, w3, eos]
    # decoder_input_ids:  [bos_id, w1, w2, w3]
    # labels:             [w1, w2, w3, eos]
    decoder_input_ids = [
        [bos_id] + [seq_id for seq_id in seq if seq_id != eos_id] for seq in input_ids
    ]

    # Update model_inputs
    model_inputs["labels"] = input_ids
    model_inputs["decoder_input_ids"] = decoder_input_ids

    model_inputs.pop("attention_mask", None)

    return model_inputs


# Select a subset of the training set
wmt14_subset = wmt14["train"].select(range(128))

# Now apply your preprocessing function to just those 128 examples
tokenized_wmt14_subset = wmt14_subset.map(
    preprocess_function, batched=True, remove_columns=wmt14["train"].column_names
)

# Convert to torch for consistency
tokenized_wmt14_subset.set_format(
    type="torch", columns=["input_ids", "decoder_input_ids", "labels"]
)

print(
    f"Checking if the tokenized_wmt14 dataset is cached correctly: {tokenized_wmt14_subset.cache_files}"
)


# Now you can print out some examples to visually inspect them
for i in range(3):
    print("Example", i)
    print("Input IDs:", tokenized_wmt14_subset[i]["input_ids"])
    print("Decoder Input IDs:", tokenized_wmt14_subset[i]["decoder_input_ids"])
    print("Labels:", tokenized_wmt14_subset[i]["labels"])

    print(
        f"decoded_input size: {tokenized_wmt14_subset[i]['input_ids'].size()}",
        "Decoded input:",
        tokenizer.decode(tokenized_wmt14_subset[i]["input_ids"]).replace(" <pad>", ""),
    )
    print(
        f"decoded_decoder_input size: {tokenized_wmt14_subset[i]['decoder_input_ids'].size()}",
        "Decoded decoder input:",
        tokenizer.decode(tokenized_wmt14_subset[i]["decoder_input_ids"])
        .replace(" <pad>", "")
        .replace("<pad>", ""),
    )
    # print(
    #     f"decoded_labels size: {tokenized_wmt14_subset[i]['labels'].size()}",
    #     "Decoded labels:",
    #     tokenizer.decode(tokenized_wmt14_subset[i]["labels"]).replace(" <pad>", ""),
    # )
    print()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Checking if the tokenized_wmt14 dataset is cached correctly: [{'filename': '/root/.cache/huggingface/datasets/wmt14/de-en/0.0.0/b199e406369ec1b7634206d3ded5ba45de2fe696/cache-cf35eeca5aedf53a.arrow'}]
Example 0
Input IDs: tensor([35999,     9,  4371, 25478,     0, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100])
Decoder Input IDs: tensor([58101,   465, 32932,     7,     4,  6274, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100])
Labels: tensor([  465, 32932,     7,     4,  6274,     0, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
   

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4117: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [14]:
from torch.utils.data import DataLoader
from torch.optim import Adam, lr_scheduler
from torch.nn import CrossEntropyLoss
# from data.tokenizer import tokenized_wmt14_subset, tokenizer

data_loader = DataLoader(tokenized_wmt14_subset, batch_size=16, shuffle=True)
model = Transformer(
    batch_size=16,
    seq_len=32,
    d_model=64,
    # +1 for bos token
    input_vocab_size=tokenizer.vocab_size + 1,
    output_vocab_size=tokenizer.vocab_size + 1,
    dropout_p=0.1,
)

d_model = 64


def lr_lambda(step_num):
    if step_num == 0:
        step_num = 1
    return d_model ** (-0.5) * step_num ** (-0.5)


trainer_args = {
    "lr": 1.0,
    "optimizer": Adam,
}

scheulder = {"scheduler": lr_scheduler.LambdaLR, "lr_lambda": lr_lambda}
trainer = Trainer(
    model=model, loss_func=CrossEntropyLoss(), scheduler=scheulder, **trainer_args
)

trainer.train(
    data_loader=data_loader, n_epochs=200, pad_token_id=tokenizer.pad_token_id
)
# trainer.run_one_epoch(
#     data_loader=data_loader, verbose=True, pad_token_id=tokenizer.pad_token_id
# )

Epoch:     20  Loss:  0.203  in   2.7 min
Epoch:     40  Loss:  0.149  in   5.5 min
Epoch:     60  Loss:  0.130  in   8.2 min
Epoch:     80  Loss:  0.118  in  10.9 min
Epoch:    100  Loss:  0.105  in  13.6 min
Epoch:    120  Loss:  0.098  in  16.4 min
Epoch:    140  Loss:  0.087  in  19.2 min
Epoch:    160  Loss:  0.062  in  21.9 min
Epoch:    180  Loss:  0.030  in  24.7 min
Epoch:    200  Loss:  0.008  in  27.4 min


[0.5990945361554623,
 0.3460991494357586,
 0.2838614545762539,
 0.2706680204719305,
 0.2649609725922346,
 0.26009259931743145,
 0.2560202553868294,
 0.25204641185700893,
 0.24822290055453777,
 0.24444864504039288,
 0.24056270718574524,
 0.2365652173757553,
 0.23246987722814083,
 0.22829570434987545,
 0.2240456771105528,
 0.2197423279285431,
 0.21541789919137955,
 0.21109434589743614,
 0.20680331997573376,
 0.20257563143968582,
 0.19843983463943005,
 0.19442328438162804,
 0.19054811634123325,
 0.1868303120136261,
 0.18328220024704933,
 0.17991124279797077,
 0.1767191793769598,
 0.17370428144931793,
 0.17086176574230194,
 0.168183920904994,
 0.16566254943609238,
 0.16328847780823708,
 0.16105248034000397,
 0.15894616954028606,
 0.1569603830575943,
 0.155086949467659,
 0.1533187571913004,
 0.15164792351424694,
 0.1500674020498991,
 0.14857107773423195,
 0.14715373516082764,
 0.14580910559743643,
 0.1445323694497347,
 0.14331951458007097,
 0.1421651765704155,
 0.14106592535972595,
 0.14001

## Using GPU to Train

In [9]:
import math
import torch
import numpy as np


def softmax(x: torch.Tensor, d: int):
    return torch.nn.functional.softmax(x, dim=d)


class FeedForward(torch.nn.Module):
    def __init__(
        self,
        d_model: int = 512,
        hidden_size: list[tuple[int]] = None,
        activation: torch.nn.Module = torch.nn.ReLU(),
    ):
        if hidden_size is None:
            hidden_size = [(d_model, 2048), (2048, d_model)]

        assert len(hidden_size) > 0, "hidden_size must be greater than 0"
        assert (
            hidden_size[0][0] == d_model and hidden_size[-1][1] == d_model
        ), "input and output dimensions must equal d_model"

        super().__init__()

        layers = []
        n = len(hidden_size)
        for i in range(n):
            layers.append(torch.nn.Linear(*hidden_size[i]))

            if i < n - 1:
                layers.append(activation)

        self.net = torch.nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        return self.net(x)


def attention(
    q: torch.Tensor,
    k: torch.Tensor,
    v: torch.Tensor,
    pad_mask: torch.Tensor,
    mask: torch.Tensor = None,
):
    x = torch.matmul(q, torch.transpose(k, 1, 2))

    x = x / math.sqrt(q.size(2))

    if mask is not None:
        mask = mask.to(x.device)
        x = x + mask

    pad_mask = pad_mask.to(x.device)
    x = x + pad_mask

    return torch.matmul(softmax(x, d=2), v)


class MultiHead(torch.nn.Module):
    def __init__(self, h: int = 8, d_model: int = 512, mask: torch.Tensor = None):
        assert d_model % h == 0, "d_model must be divisible by h"
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // h
        self.d_v = self.d_k
        self.h = h
        self.mask = mask

        self.w_q = torch.nn.Linear(d_model, d_model)
        self.w_k = torch.nn.Linear(d_model, d_model)
        self.w_v = torch.nn.Linear(d_model, d_model)
        self.w_o = torch.nn.Linear(d_model, d_model)

    def forward(
        self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, pad_mask: torch.Tensor
    ):
        batch_size, seq_len, _ = q.size()
        q_h = self.w_q(q)
        k_h = self.w_k(k)
        v_h = self.w_v(v)

        q_h = q_h.reshape(batch_size, seq_len, self.h, self.d_k).transpose(1, 2)
        k_h = k_h.reshape(batch_size, seq_len, self.h, self.d_k).transpose(1, 2)
        v_h = v_h.reshape(batch_size, seq_len, self.h, self.d_v).transpose(1, 2)

        q_h = q_h.reshape(batch_size * self.h, seq_len, self.d_k)
        k_h = k_h.reshape(batch_size * self.h, seq_len, self.d_k)
        v_h = v_h.reshape(batch_size * self.h, seq_len, self.d_v)

        scores = attention(q_h, k_h, v_h, pad_mask, mask=self.mask)

        scores = scores.reshape(batch_size, self.h, seq_len, self.d_v).transpose(1, 2)
        scores = scores.reshape(batch_size, seq_len, self.h * self.d_v)

        return self.w_o(scores)


def dropout(x: torch.Tensor, p: float = 0.1):
    assert 0 <= p <= 1, "p must be a probability"

    if p == 1:
        return torch.zeros_like(x)

    mask = torch.full_like(x, 1 - p)
    mask = torch.bernoulli(mask).to(x.device)
    x = x * mask

    return x * (1 / (1 - p))


class Transformer(torch.nn.Module):
    def __init__(
        self,
        batch_size: int,
        seq_len: int,
        d_model: int = 512,
        n: int = 6,
        number_of_heads: int = 8,
        input_vocab_size: int = 37000,
        output_vocab_size: int = 37000,
        dtype: torch.dtype = torch.float32,
        dropout_p: float = None,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.n = n
        self.d_model = d_model
        self.number_of_heads = number_of_heads
        self.vocab_size = output_vocab_size
        self.dtype = dtype
        self.dropout_p = dropout_p

        self.register_buffer(
            "pe",
            torch.from_numpy(positional_encodings(seq_len, d_model))
            .to(dtype)
            .unsqueeze(0),
        )
        self.register_buffer("attention_mask", get_attention_mask(seq_len, dtype=dtype))

        self.linear = torch.nn.Linear(d_model, output_vocab_size)
        self.input_embedding = torch.nn.Embedding(
            num_embeddings=input_vocab_size, embedding_dim=d_model
        )
        self.output_embedding = torch.nn.Embedding(
            num_embeddings=output_vocab_size, embedding_dim=d_model
        )

        self.encoders = torch.nn.ModuleList(
            Encoder(number_of_heads=number_of_heads, d_model=d_model)
            for _ in range(self.n)
        )

        self.decoders = torch.nn.ModuleList(
            Decoder(
                attention_mask=self.attention_mask,
                number_of_heads=number_of_heads,
                d_model=d_model,
            )
            for _ in range(self.n)
        )

    def forward(
        self,
        encoder_x: torch.Tensor,
        decoder_x: torch.Tensor,
        apply_softmax: bool = False,
        pad_token_id: int = 0,
    ):
        encoder_x = encoder_x.to(self.pe.device)
        decoder_x = decoder_x.to(self.pe.device)

        encoder_x_pad_mask = get_pad_mask(
            encoder_x,
            h=self.number_of_heads,
            batch_size=self.batch_size,
            seq_len=self.seq_len,
            pad_token_id=pad_token_id,
        ).to(self.pe.device)
        decoder_x_pad_mask = get_pad_mask(
            decoder_x,
            h=self.number_of_heads,
            batch_size=self.batch_size,
            seq_len=self.seq_len,
            pad_token_id=pad_token_id,
        ).to(self.pe.device)

        encoder_x = (
            self.input_embedding(encoder_x) * math.sqrt(self.d_model) + self.pe
        )
        decoder_x = (
            self.output_embedding(decoder_x) * math.sqrt(self.d_model) + self.pe
        )

        for encoder in self.encoders:
            encoder_x = encoder(encoder_x, encoder_x_pad_mask, dropout_p=self.dropout_p)

        for decoder in self.decoders:
            decoder_x = decoder(
                decoder_x,
                encoder_x,
                decoder_pad_mask=decoder_x_pad_mask,
                encoder_pad_mask=encoder_x_pad_mask,
                dropout_p=self.dropout_p,
            )

        return (
            softmax(self.linear(decoder_x), d=2)
            if apply_softmax
            else self.linear(decoder_x)
        )


In [5]:
import time
import numpy as np
import torch
import os


def save(filename, **kwargs):
    """
    Save a pytorch object to file
    See: https://pytorch.org/tutorials/beginner/saving_loading_models.html

    Arguments:
        filename: the file in which to save the object

    Possible keyword arguments (kwargs):
        epoch: the epoch so far if training
        model_state_dict: a model's state
        opt_state_dict: an optimizer's state, if training
    """
    msg = f"{filename} exists: delete it first to replace it."
    assert not os.path.exists(filename), msg
    torch.save(kwargs, filename)


def load(filename):
    """
    Load a pytorch object from a given filename
    See: https://pytorch.org/tutorials/beginner/saving_loading_models.html

    Arguments:
        filename: the file from which to load the object
    """
    return torch.load(filename)


class Trainer:
    def __init__(self, optimizer, model, loss_func, scheduler=None, device="cpu", **kwargs):
        """
        Initialize the optimizer for the model, using any necessary kwargs
        Save the model and loss function for later calculation
        """
        self.device = device
        self.optimizer = optimizer(model.parameters(), **kwargs)
        self.scheduler = scheduler["scheduler"](
            self.optimizer, lr_lambda=scheduler["lr_lambda"]
        ) if scheduler else None
        self.model = model.to(self.device)
        self.loss_func = loss_func
        self.epoch = 0
        self.start_time = None

    def run_one_batch(self, encoder_x, decoder_x, y, train=True, pad_token_id=0):
        """
        Run self.model on one batch of data, using `self.loss_func` to compute the model's loss.
        """
        # Move data to device
        encoder_x = encoder_x.to(self.device)
        decoder_x = decoder_x.to(self.device)
        y = y.to(self.device)

        if train:
            self.optimizer.zero_grad()

        outputs = self.model(encoder_x, decoder_x, pad_token_id=pad_token_id)
        loss = self.loss_func(outputs.transpose(1, 2), y)

        if train:
            loss.backward()
            self.optimizer.step()
            if self.scheduler:
                self.scheduler.step()

        return loss.detach().cpu().numpy()

    def run_one_epoch(self, data_loader, train=True, verbose=False, pad_token_id=0):
        """
        Train one epoch, a batch at a time, using self.run_one_batch.
        """
        np.random.seed(0)
        torch.manual_seed(0)

        if self.start_time is None:
            self.start_time = time.time()

        epoch_size = 0
        total_loss = 0
        for batch_idx, batch_data in enumerate(data_loader):
            encoder_x, decoder_x, y = (
                batch_data["input_ids"],
                batch_data["decoder_input_ids"],
                batch_data["labels"],
            )
            epoch_size += encoder_x.size(0)
            loss = self.run_one_batch(
                encoder_x, decoder_x, y, train=train, pad_token_id=pad_token_id
            )
            total_loss += loss

        avg_loss = total_loss / epoch_size

        if verbose:
            epoch = self.epoch + 1
            duration = (time.time() - self.start_time) / 60
            log = [f"Epoch: {epoch:6d}" if train else "Eval:" + " " * 8]
            log.extend(
                [
                    f"Loss: {avg_loss:6.3f}",
                    f"in {duration:5.1f} min",
                ]
            )
            print("  ".join(log))

        return avg_loss

    def train(self, data_loader, n_epochs, train=True, report_every=None, pad_token_id=0):
        """
        Run the model for `n_epochs` epochs on the data in `data_loader`.
        """
        self.start_time = time.time()

        if report_every is None:
            report_every = max(1, n_epochs // 10)

        losses = []
        for i in range(n_epochs):
            verbose = ((i + 1) % report_every) == 0
            loss = self.run_one_epoch(
                data_loader, train=train, verbose=verbose, pad_token_id=pad_token_id
            )
            losses.append(loss)
            if train:
                self.epoch += 1

        return losses

    def eval(self, data_loader):
        """
        Helper function to run through the data loader once and just compute the loss.
        """
        return self.train(data_loader, 1, train=False, report_every=1)

    def save_trainer(self, filename):
        """
        Save this Trainer to file.
        """
        save(
            filename,
            epoch=self.epoch,
            model_state_dict=self.model.state_dict(),
            optimizer_state_dict=self.optimizer.state_dict(),
        )

    def load_trainer(self, filename):
        """
        Load this Trainer from file.
        """
        checkpoint = load(filename)
        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        self.epoch = checkpoint["epoch"]
        self.model.to(self.device)
        self.model.train()


In [10]:
from torch.utils.data import DataLoader
from torch.optim import Adam, lr_scheduler
from torch.nn import CrossEntropyLoss

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create DataLoader
data_loader = DataLoader(tokenized_wmt14_subset, batch_size=16, shuffle=True)

# Initialize the model and move it to the appropriate device
model = Transformer(
    batch_size=16,
    seq_len=32,
    d_model=64,
    # +1 for bos token
    input_vocab_size=tokenizer.vocab_size + 1,
    output_vocab_size=tokenizer.vocab_size + 1,
    dropout_p=0.1,
).to(device)  # Move model to device

# Model dimensions
d_model = 64

# Learning rate lambda function
def lr_lambda(step_num):
    if step_num == 0:
        step_num = 1
    return d_model ** (-0.5) * step_num ** (-0.5)

# Trainer arguments
trainer_args = {
    "lr": 1.0,
    "optimizer": Adam,
}

# Learning rate scheduler
scheduler = {"scheduler": lr_scheduler.LambdaLR, "lr_lambda": lr_lambda}

# Initialize the Trainer with the device
trainer = Trainer(
    model=model,
    loss_func=CrossEntropyLoss(),
    scheduler=scheduler,
    device=device,  # Pass the device to Trainer
    **trainer_args,
)

# Start training
trainer.train(
    data_loader=data_loader,
    n_epochs=200,
    pad_token_id=tokenizer.pad_token_id,
)


Epoch:     20  Loss:  0.273  in   0.2 min
Epoch:     40  Loss:  0.273  in   0.4 min
Epoch:     60  Loss:  0.273  in   0.6 min
Epoch:     80  Loss:  0.272  in   0.8 min
Epoch:    100  Loss:  0.272  in   1.0 min
Epoch:    120  Loss:  0.269  in   1.2 min
Epoch:    140  Loss:  0.263  in   1.4 min
Epoch:    160  Loss:  0.248  in   1.6 min
Epoch:    180  Loss:  0.184  in   1.8 min
Epoch:    200  Loss:  0.121  in   2.0 min


[0.5565645806491375,
 0.3105028010904789,
 0.28261831775307655,
 0.27772708237171173,
 0.27665937319397926,
 0.27443154342472553,
 0.2741149440407753,
 0.2738286331295967,
 0.2736268937587738,
 0.27355015464127064,
 0.273493692278862,
 0.2734227627515793,
 0.27335039898753166,
 0.2733312267810106,
 0.27330354787409306,
 0.2732666824012995,
 0.27324438467621803,
 0.2732210773974657,
 0.27319613099098206,
 0.2731740493327379,
 0.27315371856093407,
 0.27313463762402534,
 0.27311631850898266,
 0.2730988934636116,
 0.2730824612081051,
 0.2730666249990463,
 0.27305140905082226,
 0.2730368487536907,
 0.27302277088165283,
 0.273009130731225,
 0.27299596555531025,
 0.272983193397522,
 0.27297077141702175,
 0.2729587461799383,
 0.272946959361434,
 0.2729355115443468,
 0.27292428724467754,
 0.27291329950094223,
 0.2729025688022375,
 0.27289201505482197,
 0.272881630808115,
 0.2728713620454073,
 0.27286130748689175,
 0.2728513702750206,
 0.27284157276153564,
 0.2728318851441145,
 0.272822326049208